In [ ]:
import torch
import unsloth
from transformers import (
    TrainingArguments,
    BitsAndBytesConfig
)
from trl import SFTTrainer
from unsloth import FastLanguageModel
from datasets import Dataset
import pandas as pd

In [ ]:
# Step 1: Configure Quantization for Memory Efficiency
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
# Step 2: Load Model and Tokenizer
model_name = "unsloth/DeepSeek-R1-Distill-Qwen-1.5B-unsloth-bnb-4bit"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name,
    max_seq_length=1024,
    quantization_config=bnb_config,
    dtype=None,
    load_in_4bit=True,
)
tokenizer.pad_token = "right"
EOS_TOKEN = tokenizer.eos_token

In [ ]:
# Step 3: LoRa Configuration
model = FastLanguageModel.get_peft_model(
    model,
    r=8,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None
)

In [ ]:
#Step 4: Dataset preparation

prompt_style = """
### Question:
{}
### Response:
{}
"""
#  Create prompts
def format_function(examples):
    return {
        "prompt": [f"User {uid} rated Item {iid} {rat}/5. Generate a detailed review:" 
                  for uid, iid, rat in zip(examples["userid"], examples["itemid"], examples["rating"])],
        "completion": examples["reviewText"]
    }
def formatting_prompts_func(examples):
    prompts = examples["prompt"]
    completions = examples["completion"]
    texts = []
    for prompt,completion in zip(prompts, completions):
        text = prompt_style.format(prompt, completion) + EOS_TOKEN
        texts.append(text)
    return {
        "text": texts,
    }

# Load Dataset
df = pd.read_csv("test_review_small.csv")
dataset = Dataset.from_pandas(df)
dataset = dataset.train_test_split(test_size=0.02)

dataset = dataset.map(
    format_function,
    remove_columns=["userid", "itemid", "rating", "reviewText"],
    batched=True,
    batch_size=1000
)

dataset=dataset.map(
    formatting_prompts_func,
    remove_columns=["prompt", "completion"],
    batched=True,
    batch_size=1000
)

# Access train and test splits
train_dataset = dataset['train']
test_dataset = dataset['test']

# Print dataset sizes and samples
print("Train size:", len(train_dataset))
print("Test size:", len(test_dataset))

In [ ]:
#Step5 : Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    learning_rate=2e-5,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
    fp16=True,
    report_to="none",
    gradient_accumulation_steps=1
)

In [ ]:
# Step 6: Training the model
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset["train"],
    dataset_text_field="text",
    max_seq_length=2048,
    dataset_num_proc=8,
    packing=True,
    args=training_args,
)
# Train the model
trainer_stats = trainer.train()

In [ ]:
# Step 7: Save Model
trainer.save_model("final_model")
tokenizer.save_pretrained("final_model")